# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
import csv
weather_df = pd.read_csv('../output_data/cities.csv')

In [77]:
weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.15,-70.92,53.60,50,40,33.33,CL,1580239820
1,1,shimoda,34.67,138.95,53.60,93,90,6.93,JP,1580240039
2,2,hilo,19.73,-155.09,72.00,83,90,11.41,US,1580240037
3,3,barrow,71.29,-156.79,-27.40,67,1,4.70,US,1580240040
4,4,vestmannaeyjar,63.44,-20.27,34.92,61,0,3.42,IS,1580240040


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [78]:
#configure gmaps
gmaps.configure(api_key=g_key)


In [79]:
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()
#set max intensity to 100 as it is max for humidity
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius = 1)

#adding heatmap layer
fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [80]:
seventy_df = weather_df[weather_df['Max Temp']> 70]
eighty_df = seventy_df[seventy_df['Max Temp']<80]
wind_df = eighty_df[eighty_df['Wind Speed']<10]
ideal_df = wind_df[wind_df['Cloudiness']==0]
ideal_df.dropna()
ideal_df.head(13)



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,chuy,-33.70,-53.46,77.54,50,0,8.03,UY,1580240040
85,85,saint-pierre,-21.34,55.48,73.40,73,0,5.82,RE,1580240048
160,160,bumba,2.18,22.47,77.07,38,0,1.54,CD,1580240056
222,222,kidal,18.44,1.41,74.46,9,0,8.99,ML,1580240062
260,260,yarada,17.65,83.27,75.20,88,0,2.24,IN,1580240066
278,278,ternate,0.80,127.40,73.47,90,0,2.35,ID,1580240029
298,298,moshupa,-24.78,25.42,73.40,69,0,8.05,BW,1580240071
376,376,ankazoabo,-22.28,44.52,70.75,87,0,8.93,MG,1580240107
377,377,nayudupeta,13.90,79.90,70.74,87,0,7.05,IN,1580240108
398,398,port augusta,-32.50,137.77,73.99,87,0,4.70,AU,1580240110


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [81]:
hotel_df = ideal_df
#adding hotel name
hotel_df['Hotel Name']= ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
lats = hotel_df["Lat"].tolist()
lng = hotel_df["Lng"].tolist()

target_hotel = "hotel"
radius = 10000000

countNum = -1

for cities in hotel_df.iterrows():
    
    countNum = countNum +1
    try:
        params = {
            "location":f"{lats[countNum], lng[countNum]}",
            "types": target_hotel,
            "radius": radius,
            "key": g_key
        }

        response = requests.get(base_url + target_city, params=params).json()
        hotel_df["Hotel Name"].iloc[countNum] = response["results"][1]["name"]
    except:
        hotel_df["Hotel Name"].iloc[countNum] = "Hotel not found"
    continue
    
hotel_df
    
    

C:\Users\balho\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\balho\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,chuy,-33.70,-53.46,77.54,50,0,8.03,UY,1580240040,Hotel not found
85,85,saint-pierre,-21.34,55.48,73.40,73,0,5.82,RE,1580240048,Hotel not found
160,160,bumba,2.18,22.47,77.07,38,0,1.54,CD,1580240056,Hotel not found
222,222,kidal,18.44,1.41,74.46,9,0,8.99,ML,1580240062,Hotel not found
260,260,yarada,17.65,83.27,75.20,88,0,2.24,IN,1580240066,Hotel not found
278,278,ternate,0.80,127.40,73.47,90,0,2.35,ID,1580240029,Hotel not found
298,298,moshupa,-24.78,25.42,73.40,69,0,8.05,BW,1580240071,Hotel not found
376,376,ankazoabo,-22.28,44.52,70.75,87,0,8.93,MG,1580240107,Hotel not found
377,377,nayudupeta,13.90,79.90,70.74,87,0,7.05,IN,1580240108,Hotel not found
398,398,port augusta,-32.50,137.77,73.99,87,0,4.70,AU,1580240110,Hotel not found


In [82]:
hotel_df = ideal_df
#adding hotel name
hotel_df['Hotel Name']= ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
lats = hotel_df["Lat"].tolist()
lng = hotel_df["Lng"].tolist()


params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
    for index, row in hotel_df.iterrows():
    #get lat, lng from df
       lat = row["Lat"]
        lng = row["Lng"]

    # change location each iteration while leaving original params in place
    ##params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
hotel_name = hotel_name.json()
print(json.dumps(hotel_name, indent=4, sort_keys=True)

IndentationError: unexpected indent (<ipython-input-82-f7fdd05b9a18>, line 16)

In [90]:
#put all of the latitude and longitude values in a list
lat = hotel_df["Lat"].tolist()
lng = hotel_df["Lng"].tolist()

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
target_type = "hotel"
radius = 5000

countNum = -1
#create a for loop that will loop through the rows to get the hotel names
#for the desired cities. 
for index in hotel_df.iterrows():
    countNum = countNum + 1
    try:
        params = {
            "location": f"{lat[countNum]}, {long[countNum]}",
            "types": target_type,
            "radius": radius,
            "key": g_key
        }
        
        
        response = requests.get(base_url + target_city, params=params).json()
        hotel_df["Hotel Name"].iloc[countNum] = response["results"][1]["name"]
    except:
        hotel_df["Hotel Name"].iloc[countNum] = "Hotel not found"
    continue
hotel_df

C:\Users\balho\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,chuy,-33.70,-53.46,77.54,50,0,8.03,UY,1580240040,Hotel not found
85,85,saint-pierre,-21.34,55.48,73.40,73,0,5.82,RE,1580240048,Hotel not found
160,160,bumba,2.18,22.47,77.07,38,0,1.54,CD,1580240056,Hotel not found
222,222,kidal,18.44,1.41,74.46,9,0,8.99,ML,1580240062,Hotel not found
260,260,yarada,17.65,83.27,75.20,88,0,2.24,IN,1580240066,Hotel not found
278,278,ternate,0.80,127.40,73.47,90,0,2.35,ID,1580240029,Hotel not found
298,298,moshupa,-24.78,25.42,73.40,69,0,8.05,BW,1580240071,Hotel not found
376,376,ankazoabo,-22.28,44.52,70.75,87,0,8.93,MG,1580240107,Hotel not found
377,377,nayudupeta,13.90,79.90,70.74,87,0,7.05,IN,1580240108,Hotel not found
398,398,port augusta,-32.50,137.77,73.99,87,0,4.70,AU,1580240110,Hotel not found


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                            hotel)

# Display Map